In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import numpy as np
import json

from collections import defaultdict

from datasets import Dataset

from utils.helpers import find_root_dir

from access_tokens import HF_WRITE_TOKEN

In [ ]:
ARDT_DIR = find_root_dir()
DIR = ARDT_DIR + "/datasets-raw/arrl-raw-data/"

In [ ]:
filepath = f"{DIR}/arrl_pr_raw_dataset-HalfCheetah-v4-v1.json"

with (open(filepath, "rb")) as f:
    data = json.load(f)

In [ ]:
transformed_data = []

for k, v in data.items():
    pad = len(v) % 1000
    if pad > 0:
        v = v[:-pad]
    sublists = [v[i:i + 1000] for i in range(0, len(v), 1000)]
    transformed_data.extend(sublists)

len(transformed_data)

In [ ]:
# convert list of dictionaries into dictionaries of lists
trajectories = []
for t in transformed_data:
    traj = defaultdict(list)
    for p in t:
        traj['observations'].append(p['state'])
        traj['pr_actions'].append(p['pr_action'])
        traj['adv_actions'].append(p['adv_action'])
        traj['rewards'].append(float(p['reward']))
        traj['dones'].append(bool(p['done']))
    trajectories.append(traj)

len(trajectories)

In [ ]:
d = defaultdict(list)
for t in trajectories:
    d['observations'].append(t['observations'])
    d['pr_actions'].append(t['pr_actions'])
    d['adv_actions'].append(t['adv_actions'])
    d['rewards'].append(t['rewards'])
    d['dones'].append(t['dones'])

ds = Dataset.from_dict(d)
print(ds)

In [ ]:
ds.save_to_disk(f'{ARDT_DIR}/datasets/arrl_prmdp_train_halfcheetah_v1')